# ZTF Events and Observations

The goal of this notebook is to estimate the number of microlensing events we can expect ZTF to discover during the lifetime of a Key Project. 

This estimate is based on the catalog of 60 events published by Rodriguez et al. 2022 2022ApJ...927..150R.  

In [66]:
from os import path
import csv
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table, Column
from astropy.io import fits
import numpy as np
import h5py

In [67]:
def read_event_list(event_file_path):
    data = []
    with open(event_file_path, 'r', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            s = SkyCoord(row[1], row[2], frame='icrs', unit=(u.hourangle, u.deg))
            data.append(row + [s])
    data = np.array(data)

    event_table = Table(
                        [Column(data=data[:,0], name='Event'),
                        Column(data=data[:,1], name='RA'),
                        Column(data=data[:,2], name='Dec'),
                        Column(data=data[:,3], name='Coordinate')]
                        )

    return event_table

Reading in this list of events, we can calculate the number of lensing events detected each year.

In [68]:
event_file_path = 'ztf_event_list.csv'
ztf_events = read_event_list(event_file_path)


In [69]:
def count_events_per_year(event_set):
    events_per_year = {}
    uid_events = 0
    for event in event_set:
        # ZTF events have been assigned names in a variety of formats, not all of which include the
        # year they were discovered.  Sigh.  Going to parse as many of them as possible.
        name = str(event['Event'])
        if name[0:3] == 'ZTF' and name[3:4] in ['1', '2']:
            year = '20' + name[3:5]
        elif name[0:2] == 'AT' and name[2:3] in ['1', '2']:
            year = name[2:6]
        else:
            uid_events += 1
            year = '9999'
        
        if year != '9999':
            if year not in events_per_year.keys():
                events_per_year[year] = 1
            else:
                events_per_year[year] += 1

    for key, value in events_per_year.items():
        print(str(key)+': '+str(value)+' events')
    print(str(uid_events)+' events with year unidentified')

Now let's take a look at the expected rate of discoveries, bearing in mind that the survey and the alert system took a while to ramp up to the best rate of discoveries. 

In [70]:
print('Events per year discovered in DR 14 data:')
count_events_per_year(ztf_events)

Events per year discovered in DR 14 data:
2019: 25 events
2020: 17 events
2018: 11 events
7 events with year unidentified


Rodriguez et al. noted 3 events showing parallax signatures, and no binaries. 

## Expected number of events within a Key Project

From this it is clear that the rate of discovery is quite variable, but we can expect between 10-25 events per year.  **We adopt a conservative estimate of 15 events/year.**  

Based on the kp_simulator simulation, LCO would be able to observe 99% of all ZTF discovered events.  Rodriguez doesn't give an estimate of the expected fraction of stellar remnant events, so we use the figure of 7.9% of events derived from the Gaia catalog.  **This implies approximately 4-5 such lenses over the Key Project duration.**  

In [83]:
n_ztf_events = 15 # Per year
frac_bh_events = 0.085
frac_stellar_events = 0.915
obs_frac = 0.99
n_years = 3

n_stellar_per_year = int(round((n_ztf_events * frac_stellar_events * obs_frac),0))
n_bh_per_year = int(round((n_ztf_events * frac_bh_events * obs_frac),0))
n_stellar = n_stellar_per_year * n_years
n_bh = n_bh_per_year * n_years

print('Number of stellar events: '+str(n_stellar_per_year)+' per year, '+str(n_stellar)+' in total')
print('Number of stellar remnant events: '+str(n_bh_per_year)+' per year, '+str(n_bh)+' in total')



Number of stellar events: 14 per year, 42 in total
Number of stellar remnant events: 1 per year, 3 in total


In [93]:
frac_binary_events = 0.1
n_binary_events_obs = int(round((frac_binary_events * frac_stellar_events * n_ztf_events * obs_frac),0))
print('We expect ~'+str(round(n_binary_events_obs,0))+' binary events to be observable during the lifetime of the Key Project')


We expect ~1 binary events to be observable during the lifetime of the Key Project


## Observing Time Required

We can use the simulations of the Key Project observing strategy to estimate the total time required to observe ZTF events. The kp_simulator notebook generated a simulated set of 1000 events, and stored the simulated data products, which we read in here.  

In [94]:
def read_sim_event_table(file_path):
    hdul = fits.open(file_path)
    cols = hdul[1].columns
    data = hdul[1].data
    column_list = []
    for col in cols:
        if col.name == 'EventID':
            dtype = 'str'
        elif col.name in ['HEALpixel', 'nvisits']:
            dtype = 'int'
        else:
            dtype = 'float'
        column_list.append(Column(name=col.name, data=data[col.name], dtype=dtype))
    
    return Table(column_list)

In [95]:
def read_lco_sim_data(file_path, n_events):
    f = h5py.File(file_path, "r")
    dataset = {}
    for eventid in f.keys():
        dataset[eventid] = f[eventid][:]
    
    return dataset

In [96]:
sim_lco_lc_file = '../simulated_lco_lightcurves_ztf_events.hdf5'
dataset = read_lco_sim_data(sim_lco_lc_file, 1000)

In [97]:
sim_data_file = '../sim_ztf_events_table.fits'
events_table = read_sim_event_table(sim_data_file)

In [98]:
events_table

EventID,HEALpixel,RA_deg,Dec_deg,baseline_mag,t0,u0,tE,rho,piEN,piEE,nvisits_g,nvisits_r,nvisits_i,binary_lens
str14,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ZTFSim00001,20548,276.328125,9.594066619873047,20.21500015258789,2461023.75,0.08381129801273346,44.285274505615234,0.0038371486589312553,0.4844960570335388,-0.019202303141355515,30.0,190.0,3.0,0.0
ZTFSim00002,23359,268.59375,2.9855055809020996,22.21500015258789,2460417.0,0.031126264482736588,63.99932861328125,0.024312041699886322,-0.592085063457489,-0.3102404773235321,186.0,462.0,1.0,0.0
ZTFSim00003,26174,267.890625,-3.5833194255828857,21.21500015258789,2460180.0,0.2559991776943207,147.99752807617188,0.024010464549064636,0.9258144497871399,-0.2086178958415985,160.0,405.0,0.0,0.0
ZTFSim00004,8535,302.34375,41.014495849609375,22.21500015258789,2460855.25,-0.40238818526268005,60.37038803100586,0.007864141836762428,-0.26124289631843567,0.07433734089136124,840.0,1356.0,128.0,0.0
ZTFSim00005,16712,281.25,18.83940315246582,22.21500015258789,2460897.25,-0.00892803817987442,59.62217712402344,0.006026129238307476,1.7151165008544922,-0.15724730491638184,240.0,563.0,12.0,0.0
ZTFSim00006,26687,269.296875,-4.780191898345947,21.21500015258789,2460726.0,0.2368600219488144,8.464742660522461,0.0003753613855224103,0.130042165517807,0.4769270122051239,162.0,444.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTFSim00994,23114,284.765625,3.5833194255828857,21.21500015258789,2461148.5,0.2736056447029114,42.461952209472656,0.01208465825766325,1.0382159948349,-0.44371601939201355,281.0,585.0,105.0,0.0
ZTFSim00995,22343,279.84375,5.379377841949463,22.21500015258789,2460198.25,-0.37588462233543396,9.440351486206055,0.002263339702039957,-0.3324893116950989,0.04379330575466156,463.0,1047.0,105.0,0.0


Selecting the long timescale events with tE > 100d, we can use the simulated LCO lightcurves produced by the Key Project observing strategy, to estimate the typical number of visits to target that we can expect.  We can also use the same data to estimate the total exposure time per target, since the simulation includes target brightness as a function of time.  In order to estimate realistic exposure times on LCO 1m telescopes for targets at these magnitudes, we use the following empirical function, which was derived from previous LCO 1m photometry. 

In [99]:
def ETC(magin):
    """
    This function calculates the required exposure time
    for a given iband magnitude (e.g. OGLE I which also
    roughly matches SDSS i) based on a fit to the empiric
    LCO photometry.  Adapted from code by Markus Hundertmark.
    """
    
    mag = np.zeros(len(magin))
    mag[:] = magin
    
    # Cap the exposures at the bright end to minimise scintillation noise (min 5s)
    idx = np.where(magin < 14.7)
    mag[idx] = 14.7

    lrms = 0.14075464 * mag * mag - 4.00137342 * mag + 24.17513298
    snr = 1.0 / np.exp(lrms)

    # target 4% -> snr 25
    exptime = np.round((25. / snr)**2 * 300.,1)
    
    #Avoid exposures longer than 5min:
    idx = np.where(exptime > 300.0)
    exptime[idx] = 300.0

    return  exptime

In addition to the open-shutter time required during each visit to target, we need to account for telescope slew, instrument readout etc.  Based on the LCO instrumentation website (https://lco.global/observatory/instruments/), the overheads per frame for the LCO 1m Sinistro cameras is 28s.  So we add this to each exposure. 

In [100]:
def estimate_obs_time(event_set, events_table, dataset):
    
    lco_obs_time = []
    lco_nvisits = []
    
    for ievent in event_set:
        event_name = events_table[ievent]['EventID']
        lco_lc = dataset[event_name]
        lco_nvisits.append(len(lco_lc))

        # Calculate the exposure time per visit based on the target magnitude at the time, then 
        # add the instrumental overheads
        if len(lco_lc) > 0:
            lco_exposures = ETC(lco_lc[:,1])
            lco_exposures += 28.0
        else:
            lco_exposures = np.zeros(1)
        
        lco_obs_time.append(lco_exposures.sum())
        
    lco_nvisits = np.array(lco_nvisits)
    lco_obs_time = np.array(lco_obs_time)/3600.0

    return lco_nvisits, lco_obs_time


In [101]:
# Get the table indices of long-duration events
bh_events = np.where(events_table['tE'] > 100.0)[0]

# Estimate the number of visits per target and observing time required:
(lco_nvisits, lco_obs_time) = estimate_obs_time(bh_events, events_table, dataset)

time_per_bh_event = np.median(lco_obs_time)

print('Median number of LCO visits per event: '+str(np.median(lco_nvisits)))
print('Median total LCO exposure time per event: '+str(time_per_bh_event)+'hrs')

Median number of LCO visits per event: 495.0
Median total LCO exposure time per event: 9.322499999999998hrs


In [102]:
total_time_bh_events = n_bh * time_per_bh_event
print('The expected sample of '+str(n_bh)+' long-duration events is therefore estimated to required a total of '+str(round(total_time_bh_events,0))+'hrs')
       

The expected sample of 3 long-duration events is therefore estimated to required a total of 28.0hrs


To estimate the time required for stellar binary events, we adopt a similar approach, but select the correspondingly shorter timescale events from the simulated sample.

In [103]:
# Get the table indices of stellar events
#stellar_events = np.where(events_table['tE'] < 100.0)[0]
stellar_lenses = np.where(events_table['tE'] < 100.0)[0]
single_lenses = np.where(events_table['binary_lens'] == 0)[0]
single_stellar_lenses = list(set(stellar_lenses).intersection(set(single_lenses)))


# Estimate the number of visits per target and observing time required:
(lco_nvisits, lco_obs_time) = estimate_obs_time(single_stellar_lenses, events_table, dataset)

time_per_event = np.median(lco_obs_time)

print('Median number of LCO visits per event: '+str(np.median(lco_nvisits)))
print('Median total LCO exposure time per event: '+str(time_per_event)+'hrs')

Median number of LCO visits per event: 311.0
Median total LCO exposure time per event: 5.857166666666666hrs


In [104]:
total_time_stellar_events = n_stellar * time_per_event
print('The expected sample of '+str(n_stellar)+' stellar events is therefore estimated to required a total of '+str(round(total_time_stellar_events,0))+'hrs')


The expected sample of 42 stellar events is therefore estimated to required a total of 246.0hrs


Binary lens events are allocated an additional 12hrs of high-cadence (every 15min) observations on top of the regular monitoring.

In [105]:
add_time_binary_events = n_binary_events_obs * 12.0
print('The '+str(n_binary_events_obs)+' binary events are estimated to require an additional '+str(round(add_time_binary_events,0))+'hrs')



The 1 binary events are estimated to require an additional 12.0hrs


In [106]:
total_time = total_time_bh_events + total_time_stellar_events + add_time_binary_events
print('In total, LCO observations of ZTF targets would be expected to require: '+str(total_time)+'hrs')

In total, LCO observations of ZTF targets would be expected to require: 285.96849999999995hrs


## Observations from the 2m Network and Rapid Response Observations
The 2m network offers the MusCAT multi-channel imagers and the FLOYDS spectrographs.  

While only a few targets will be accessible to FLOYDS (r<15mag), the spectra are extremely useful to constrain the source star parameters for all categories of microlensing event.  In order to distinguish the source star spectrum from neighboring objects, spectra at multiple epochs with different lensing magnifications are required, though not at the high cadence required for photometry.  We propose to obtain 3 spectra of selected targets that reach magnitudes brighter than 15mag (where FLOYDS can deliver S/N=100 in 1hr of exposure).  

We can estimate the potential number of FLOYDS targets using the simulated LCO lightcurve dataset, and identifing the fraction of targets likely to exceed 15mag.  

In [107]:
sim_survey_lc_file = '../simulated_ztf_lightcurves.hdf5'
survey_dataset = read_lco_sim_data(sim_survey_lc_file, 1000)

In [108]:
bright_events = []
n_survey_lc = 0
for ievent in range(0,len(survey_dataset),1):
    event_name = events_table[ievent]['EventID']
    
    survey_lc = dataset[event_name]
    if len(survey_lc) > 0:
        n_survey_lc += 1
        idx = np.where(survey_lc[:,1] < 15.0)[0]
        if len(idx) > 0:
            bright_events.append(ievent)

frac_bright_events = float(len(bright_events))/float(len(events_table))
print('Fraction of events accessible to FLOYDS = '+str(frac_bright_events))
print('Number of full survey lightcurve '+str(n_survey_lc))

Fraction of events accessible to FLOYDS = 0.994
Number of full survey lightcurve 994


In [109]:
n_targets = n_bh_events + n_stellar_events
n_FLOYDS_targets = int(round(n_targets * frac_bright_events,0))
print('Number of ZTF targets likely to be accessible to FLOYDS = '+str(n_FLOYDS_targets))

Number of ZTF targets likely to be accessible to FLOYDS = 30


Since there is a FLOYDS spectrograph in both hemispheres, the 2m network can observe any target while it is visible to our target surveys.  Of the stellar and BH targets we expect per year we expect to be able to observe ~ 2 with FLOYDS.  
Each FLOYDS observation will consist of a 1hr science exposure plus the LCO-recommended set of calibration exposures: 2x60s arc frames with the HgAr and Zn lamps for wavelength calibration and 2x70s flat field exposures with the Tungsten-Halogen + Xenon lamp, since we will use the 1.2" slit due to the crowding in this field.  Calibration frames will be requested both before and after each science spectrum.  

Following the overhead calculation recommended on the FLOYDS website for a 1hr science spectrum:
Initial setup and telescope slew is 120 s. 
Target acquisition is 90 s. 
Lamp flat exposures include an overhead of 60 s.
Arc lamp exposures include an overhead of 100 s. 
All configuration changes (from flat to arc to science spectrum) require a 16 s software setup overhead. 
Readout time is 26 s. 

In [54]:
# All times in s
tel_slew = 120.0
target_acquisition = 90.0
lamp_exp_overhead = 60.0
lamp_exp = 70.0
arc_exp_overhead = 100.0
arc_exp = 60.0
software = 16.0
readout = 26.0
science_exp = 3600.0

In [55]:
time_per_FLOYDS_visit = tel_slew + target_acquisition \
                    + lamp_exp_overhead + software + lamp_exp + readout \
                    + arc_exp_overhead + software + arc_exp + readout \
                    + software + science_exp + readout \
                    + lamp_exp_overhead + software + lamp_exp + readout \
                    + arc_exp_overhead + software + arc_exp + readout
time_per_FLOYDS_visit /= 60.0

print('Time required per FLOYDS visit to 1 target = '+str(time_per_FLOYDS_visit)+'min')

Time required per FLOYDS visit to 1 target = 76.66666666666667min


Taking 3 visits per FLOYDS target therefore requires a total request of (split 50:50 North/South):

In [61]:
total_FLOYDS_time = n_FLOYDS_targets * time_per_FLOYDS_visit * 3.0
total_FLOYDS_time /= 60.0

print('Total time = '+str(total_FLOYDS_time)+'hrs')

Total time = 115.0hrs


The MuSCAT four-channel imagers are particularly important for binary lens events, as they can deliver high-cadence multi-color observations of caustic crossings, short-lived, sharp discontinuities in the lightcurves that mark the entrance and exits of caustic crossings lasting ~ 6hrs.  It is extremely important to monitor their photometry continuously for that period in order to constrain the morphology of the lightcurve, which in turn constrains the models of the caustic structure and lens-source relative trajectory.  

While almost all of the observations from this program can be made in queue-scheduled mode, the short timescale and transient nature of these features require rapid response time to ensure that we can observe as soon as they are detected by our real-time modeling system.  They also justify observing from every site in the network to ensure as continuous coverage as possible.  For this reason, we request the following time on MuSCAT and some 1m time is allocated in Rapid Response mode.

Caustic crossings are only likely in the stellar binary events we expect from ZTF.  Adopting an average duration of 6hrs per caustic entrance and exit (i.e. 12hrs per target), we request in Rapid Response mode:

In [57]:
time_caustic = 6.0 # hrs, with two caustic crossings per event
time_rr_stellar_events = n_stellar_events * 2.0 * time_caustic
print('Rapid Response Time requested for stellar binary caustic crossings = '+str(time_rr_stellar_events)+'hrs')

Rapid Response Time requested for stellar binary caustic crossings = 300.0hrs


ZTF targets are accessible to FTN/MusCAT for up to 8hrs per day, or a day fraction of:

In [58]:
vis_fraction = 8.0 / 24.0
vis_fraction

0.3333333333333333

We therefore request that this fraction of the require rapid response time from the FTN/MusCAT imagers, with the remaining rapid response time coming from the 1m/Sinistro network. 

In [59]:
time_2m_imagers = time_rr_stellar_events * vis_fraction
print('Time on 2m imagers in Rapid Response mode = '+str(time_2m_imagers))

Time on 2m imagers in Rapid Response mode = 100.0
